In [201]:
from mido import MidiFile, MidiTrack, Message
from music21 import *
from IPython.display import Image
import pandas as pd
import numpy as np
import math
import glob

In [185]:
def parse_notes(file_name, song_len=200):
    # function to take a midi file and create a dataframe with columns representing note played, duration and time
    
    # start by reading the file:
    message_strings_split = []
    mid = MidiFile(file_name) 
    for i in mid.tracks[1][2:-1]: 
        message_string = str(i)
        message_strings_split.append(message_string.split(" "))
        
    # now extract all the relevant information from the message and create a data frame:
    message_type = []
    for item in message_strings_split:
        message_type.append(item[0])
    df1 = pd.DataFrame(message_type)
    attributes = []
    for item in message_strings_split:
        attributes.append(item[1:])
    attributes_dict = [{}]    
    for item in attributes:
        for i in item:
            key, val = i.split("=")
            if key in attributes_dict[-1]:
                attributes_dict.append({})
            attributes_dict[-1][key] = val
    df2 = pd.DataFrame.from_dict(attributes_dict)
    df_complete = pd.concat([df1, df2], axis=1)
    
    # control change messages are for the pedal...let's simplify by not having those, and don't need all columns:
    df_notes = df_complete[df_complete[0] == 'note_on'].drop(columns={0,'channel'}).reset_index(drop=True)
    if 'control' in df_notes.columns:
        df_notes = df_notes.drop(columns={'control','value'})
    
    # change some of the data types:
    df_notes.time = df_notes.time.astype(float)
    df_notes.note = df_notes.note.astype(int)
    df_notes.velocity = df_notes.velocity.astype(int)
    
    # create a time elapsed attribute equal to the cumulative sum of time.
    df_notes['time_elapsed'] = df_notes.time.cumsum()
    
    # say we would work with a small note subset of the song
    # for speed we'll just work with the first bit of the song for calculating durations:
    df_song_intro = df_notes.loc[0:song_len*3].copy()
    
    # find the duration each note was played based on the stop note signals (note on with velocity == 0)
    duration = [0] * len(df_song_intro)
    for i in range(len(df_song_intro)):
        if df_notes['velocity'][i] != 0 and i < len(df_song_intro) - 1:            
            j = i + 1
            while df_song_intro['note'][j] != df_song_intro['note'][i]:
                if j >= len(df_song_intro) - 1:
                    break
                else:
                    j += 1
            duration[i] = df_song_intro['time_elapsed'][j] - df_song_intro['time_elapsed'][i]
    df_song_intro['duration'] = duration

    # now drop the "notes off" signal rows (this info is in the duration column)
    df_song_intro = df_song_intro[df_song_intro['velocity'] != 0].reset_index(drop=True)
    # simplify again to start without dynamics:
    df_song_intro = df_song_intro.drop(columns={'time','velocity'})
    
    # now formally take just the first bit of the song:
    df_first_notes = df_song_intro.loc[0:song_len-1].copy()
    if len(df_first_notes) < song_len:
        return np.zeros((1,200,3))
    
    # now, let's normalize the time elapsed and make duration a fraction of time elapsed:
    df_first_notes['time_elapsed'] -= df_first_notes['time_elapsed'][0]
    df_first_notes['duration'] /= df_first_notes['time_elapsed'][song_len-1]
    df_first_notes['time_elapsed'] /= df_first_notes['time_elapsed'][song_len-1]
    
    # finally, let's recreate the "time since last event" nature of a midi file for time_elapsed:
    time_since_last = [0] * song_len
    for i in range(1, song_len):
        time_since_last[i] = df_first_notes['time_elapsed'][i] - df_first_notes['time_elapsed'][i-1]
    df_first_notes['time_since_last'] = time_since_last
    df_first_notes = df_first_notes.drop(columns='time_elapsed')
    
    # lastly, need to normalize the notes...MIDI for piano returns 21 to 108, so:
    df_first_notes['note'] -= 20
    df_first_notes['note'] /= 88
    
    return df_first_notes

In [212]:
def recreate_midi(df_first_notes, speed=20000):
    # function to take a dataframe created by something like parse_notes() or a gan and return a midi
    
    # Can start by reverse scaling the note:
    df_reversed = df_first_notes.copy()
    df_reversed['note'] = round(df_reversed['note'] * 88 + 20)  # might want to have something more special than round()
    df_reversed.note = df_reversed.note.astype(int)
    df_reversed['velocity'] = 60  # create a uniform middling velocity

    # recreate the absolute time index and drop time_since_last (we'll recreate it with the stop signals)
    df_reversed['time_index'] = df_reversed.time_since_last.cumsum()
    df_reversed = df_reversed.drop(columns = 'time_since_last')

    # create a stop signal for each note at the appropriate time_index:
    for i in range(len(df_reversed)):
        stop_note = pd.DataFrame([[df_reversed.note[i], 0, 0, df_reversed.duration[i] + df_reversed.time_index[i]]],
                                 columns=['note', 'duration', 'velocity', 'time_index'])
        df_reversed = df_reversed.append(stop_note, ignore_index=True)
    df_reversed = df_reversed.sort_values('time_index').reset_index(drop=True)

    # recreate time_since last with the stop note signals
    df_reversed['time'] = [0] + [df_reversed.time_index[i+1] - df_reversed.time_index[i] 
                                 for i in range(len(df_reversed)-1)]
    # and now we don't need duration or time_index so can drop those
    df_reversed = df_reversed.drop(columns = {'time_index','duration'})

    # finally, we need to scale the time since last note appropriately:
    df_reversed['time'] = round(df_reversed['time'] * speed)
    df_reversed.time = df_reversed.time.astype(int)

    # finally, recreate the midi and return
    mid_remade = MidiFile()
    track = MidiTrack()
    mid_remade.tracks.append(track)
    track.append(Message('program_change', program=0, time=0))
    for i in range(len(df_reversed)):
        track.append(Message('note_on', note=df_reversed.note[i], velocity=df_reversed.velocity[i], time=df_reversed.time[i]))

    return mid_remade

In [187]:
# Parse all files into np array:
song_len = 200
all_songs = np.zeros((1,200,3))  # create a blank first "song" to just append things to uniformly in loop

for file_name in glob.glob("All_Maestro/*.midi"):
    song_notes = parse_notes(file_name, song_len)
    if not np.array_equal(song_notes, np.zeros((1,200,3))):
        transpose_notes = song_notes.copy()
        for i in range(-5,7):
            transpose_notes['note'] = song_notes['note'] + i/88
            transpose_notes['note'] = [1/88 if i <= 0 else i for i in transpose_notes['note']] # can't go below bottom A
            transpose_notes['note'] = [1 if i > 1 else i for i in transpose_notes['note']] # can't go above top C
            all_songs = np.append(all_songs, transpose_notes.to_numpy().reshape((1,200,3)), axis=0)

all_songs = np.delete(all_songs, 0, 0)  # delete that first blank song

In [216]:
np.save('All_Maestro_Parsed', all_songs) # save the parse data for later use
# all_songs = np.load('All_Maestro_Parsed.npy') # test loading of saved array

In [188]:
# can test various elements of array
all_songs.shape
# all_songs

(15300, 200, 3)

In [228]:
# test recreating a midi
test_song = pd.DataFrame(all_songs[0,:,:].reshape((200,3)), columns=["note", "duration","time_since_last"])
mid_remade = recreate_midi(test_song, 20000)
mid_remade.save('mid_test.mid')

In [229]:
# see if we can play in jupyter notebook
mf = midi.MidiFile()
mf.open('mid_test.mid')
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
s.show('midi') # note at this stage there will be no dynamics and no pedal